In [1]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import os 
import json 
import urllib
from urllib.parse import unquote


In [ ]:

def crawler_url(kpop_url_list=dict()):
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")

    driver = webdriver.chrome.webdriver.WebDriver(executable_path='../chromedriver', chrome_options=options)
    for num in range(1,6):
        url = f'https://korean.visitseoul.net/hallyu?curPage={num}&srchType=&srchOptnCode=&srchCtgry=29&sortOrder=&srchWord='
        driver.get(url)


        try:
            article_list = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'article-list-slide'))
            )
            articles = article_list.find_elements_by_tag_name("li")
            for article in articles:
                a_tag = article.find_element_by_tag_name("a")

                title = a_tag.get_attribute('title')
                href  = a_tag.get_attribute('href')
                href = urllib.parse.unquote(href)
                thumb = a_tag.find_element_by_class_name('thumb').get_attribute('style')

                kpop_url_list[title] = {
                    'href': href,
                    'thumb' : 'https://korean.visitseoul.net/' +thumb.replace('background-image: url("','').replace('");',''),
                }
                print(a_tag.get_attribute('title'))
                print(a_tag.get_attribute('href'))
                print(a_tag.find_element_by_class_name('thumb').get_attribute('style'))

        except Exception as ex: # 에러 종류
            print('에러가 발생 했습니다', ex) #
            driver.quit()

    driver.quit()
    return kpop_url_list

In [ ]:
kpop_url_list = crawler_url()
print(kpop_url_list)
with open('./kpop_url_page_fron_1_to_6.json', 'w', encoding='UTF-8') as outfile:
    json.dump(kpop_url_list, outfile, ensure_ascii=False)

In [3]:
def detail_page(url):
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")

    driver = webdriver.chrome.webdriver.WebDriver(executable_path='../chromedriver', chrome_options=options)
    driver.get(url)
    try:
        info_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'infor-element'))
        )
        category_title_selenium = info_element.find_element_by_class_name('h3')
        category_title = category_title_selenium.text
        print(category_title)
        try:
            kpopcontent, _ = KPopContent.objects.get_or_create(
                title=category_title,            
                content_type = 'tv'
            )
            print(kpopcontent)
        except:
            print('kpopcontent model make failed')
        text_area = info_element.find_element_by_class_name('text-area')
        tourist_spots = text_area.find_elements_by_class_name('cnt-blockquote')
        for spot in tourist_spots:
            spot_name = spot.find_element_by_class_name('fc-blue').text
            spot_info = spot.find_elements_by_tag_name('p')
            
            try:
                address = spot_info[0].text
            except:
                address = '없음'

            try:
                open_hour = spot_info[1].text
            except:
                open_hour = '없음'
            try:
                telphone = spot_info[2].text
            except:
                telphone = '없음'
            try:
                web_site = spot_info[3].text
            except:
                web_site = '없음'
            try:
                transportation = spot_info[4].text
            except:
                transportation = '없음'

            place,_ = Place.objects.get_or_create(
                name = spot_name,
                address = address,
                phone_number = telphone,
                open_time = open_hour,
                url = web_site,
                
            )
            KPopCategory.objects.get_or_create(
                place = place,
                kpop_content = kpopcontent,
            )
            print(place)
    except Exception as ex: # 에러 종류
        print('에러가 발생 했습니다', ex) #
        driver.quit()
    driver.quit()

In [4]:
with open('./kpop_url_page_fron_1_to_6.json', 'r') as file:
    json_data = json.load(file)
for title in json_data:
    url = json_data[title]['href']
    print(url)
    try:
        data = detail_page(url)
        
    except:
        print('다른 페이지 오류를 어떻게 잡지')
    

https://korean.visitseoul.net/hallyu/강다니엘을-더-가까이-다니티-투어_/31622


/home/kimdooh/.pyenv/versions/kopa-tour-env/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


강다니엘을 더 가까이, ‘다니티’ 투어
카테고리 mu의 제목은 강다니엘을 더 가까이, ‘다니티’ 투어 입니다
탐방코스
더 피아노
LH2
마봉 양꼬치 용산점
카페 피키파파 다니엘포레스트
https://korean.visitseoul.net/hallyu/세븐틴을-따라가는‘아주-Nice’한-캐럿투어_/31620
세븐틴을 따라가는‘아주 Nice’한 캐럿투어
카테고리 mu의 제목은 세븐틴을 따라가는‘아주 Nice’한 캐럿투어 입니다
탐방코스
청춘식당 미래소년
알로하포케 학동점
브릭캠퍼스 서울
스피닝울프 성수
https://korean.visitseoul.net/hallyu/K-Beauty-전성시대-‘뷰티룸’을-따라가는-한류-인싸-여행_/30409
K-Beauty 전성시대, ‘뷰티룸’을 따라가는 한류 인싸 여행
카테고리 mu의 제목은 K-Beauty 전성시대, ‘뷰티룸’을 따라가는 한류 인싸 여행 입니다
탐방코스
온앤더뷰티
키친랩 가로수길점
JOY187
집사의하루
https://korean.visitseoul.net/hallyu/엑소를-더-가까이--‘엑소엘-투어’_/30404
엑소를 더 가까이, ‘엑소엘 투어’
카테고리 mu의 제목은 엑소를 더 가까이, ‘엑소엘 투어’ 입니다
탐방코스
인사동촌
아비꼬 압구정로데오점
오늘 성수 맑음
샤로스톤 외대점
https://korean.visitseoul.net/hallyu/몬스타엑스의-에너지원-서울-맛집을-찾아서_/30399
몬스타엑스의 에너지원! 서울 맛집을 찾아서
카테고리 mu의 제목은 몬스타엑스의 에너지원! 서울 맛집을 찾아서 입니다
탐방코스
마라토끼
모단걸응접실 가로수길점
이모가 있는 집
족황상제 압구정로데오점
https://korean.visitseoul.net/hallyu/팔로우미11-속-오-마이-서울_/29879
<팔로우미11> 속 오 마이 서울
카테고리 mu의 제목은 <팔로우미11> 속 오 마이 서울 입니다
탐방코스
카페 레이어드
타이거슈가 홍대본점
호텐
오드갤러리
https://korean.visitseoul.net

In [32]:
celeb_list = ['강다니엘', '세븐틴', '엑소', '몬스타엑스', '방탄소년단','아미','오마이걸', '규현', '아이즈원', '뉴이스트', 'TXT', '갓세븐', 'NCT', 'BTS', '블랙핑크', '위너', '유노윤호', '산다라박', '이하이', '이수현','워너원','동할배', '빅뱅', '소년24']

In [35]:
for name in celeb_list:
    celeb, _ = Celebrity.objects.get_or_create(
        name = name        
    )
    kpopcontent_list = KPopContent.objects.filter(title__contains=celeb.name)
    for content in kpopcontent_list:
        content.celebrity = celeb
        content.save()

In [21]:
kpopcontent_list = KPopCategory.objects.filter(kpop_content__title__contains='BTS')

In [19]:

def drama_crawler_url(kpop_url_list=dict()):
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")

    driver = webdriver.chrome.webdriver.WebDriver(executable_path='../chromedriver', chrome_options=options)
    for num in range(1,6):
        
        url = f'https://korean.visitseoul.net/hallyu?curPage={num}&srchType=&srchOptnCode=&srchCtgry=30&sortOrder=&srchWord='
        driver.get(url)
        print(url)

        try:
            article_list = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'article-list-slide'))
            )
            articles = article_list.find_elements_by_tag_name("li")
            for article in articles:
                a_tag = article.find_element_by_tag_name("a")

                title = a_tag.get_attribute('title')
                href  = a_tag.get_attribute('href')
                href = urllib.parse.unquote(href)
                thumb = a_tag.find_element_by_class_name('thumb').get_attribute('style')

                kpop_url_list[title] = {
                    'href': href,
                    'thumb' : 'https://korean.visitseoul.net/' +thumb.replace('background-image: url("','').replace('");',''),
                }
                print(title)
                print(href)
                print(thumb)

        except Exception as ex: # 에러 종류
            print('에러가 발생 했습니다', ex) #
            driver.quit()

    driver.quit()
    return kpop_url_list

In [20]:
kpop_url_list = drama_crawler_url()
print(kpop_url_list)
with open('./k_drama_url_page_fron_1_to_6.json', 'w', encoding='UTF-8') as outfile:
    json.dump(kpop_url_list, outfile, ensure_ascii=False)

/home/kimdooh/.pyenv/versions/kopa-tour-env/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


https://korean.visitseoul.net/hallyu?curPage=1&srchType=&srchOptnCode=&srchCtgry=30&sortOrder=&srchWord=
드라마의 감동과 여운을 다시 한 번 더! <호텔 델루나>를 찾아서
https://korean.visitseoul.net/hallyu/호텔-델루나를-찾아서_/34539
background-image: url("/comm/getImage?srvcId=POST&parentSn=34539&fileTy=POSTTHUMB&fileNo=1&thumbTy=M");
당신도 봉준호 마니아라면 봉하이브 투어
https://korean.visitseoul.net/hallyu/봉하이브-투어_/34310
background-image: url("/comm/getImage?srvcId=POST&parentSn=34310&fileTy=POSTTHUMB&fileNo=1&thumbTy=M");
꺼지지 않는 열풍! <사랑의 불시착> 속 촬영지
https://korean.visitseoul.net/hallyu/꺼지지-않는-열풍-사랑의-불시착-속-촬영지_/34308
background-image: url("/comm/getImage?srvcId=POST&parentSn=34308&fileTy=POSTTHUMB&fileNo=1&thumbTy=M");
근대역사 골목산책 「우리슈퍼」 코스
https://korean.visitseoul.net/hallyu/근대역사-골목산책-우리슈퍼-코스_/34204
background-image: url("/comm/getImage?srvcId=POST&parentSn=34204&fileTy=POSTTHUMB&fileNo=1&thumbTy=M");
서울라이트 일상 체험 「피자시대 코스」 
https://korean.visitseoul.net/hallyu/서울라이트-일상-체험-피자시대-코스_/34203
background-image: url("/comm/getImage?srvcId=POS